In [2]:
import pandas as pd
import numpy as np
import geopandas as gpd


In [3]:
df = pd.read_csv('../Data/data_GLDAS/compiled_canada_soil_moisture.csv')

In [4]:
df.columns

Index(['time', 'lat', 'lon', 'SoilMoi0_10cm_inst', 'SoilMoi10_40cm_inst',
       'SoilMoi40_100cm_inst', 'SoilMoi100_200cm_inst'],
      dtype='object')

In [5]:
df = df.drop(['SoilMoi10_40cm_inst', 'SoilMoi40_100cm_inst', 'SoilMoi100_200cm_inst'], axis = 1)
#rename the column 'SoilMoi0_10cm_inst' to 'waterstorage'
df = df.rename(columns={'SoilMoi0_10cm_inst': 'waterstorage'})
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df['lon'], df['lat']))
gdf = gdf.set_crs(epsg=4326, inplace=True)

gdf.to_parquet('../Data/data_GLDAS/gdf_compiled_canada_soil_moisture.parquet', index=False)